# Eat Safe, Love

## Part 1: Database and Jupyter Notebook Set Up

Import the data provided in the `establishments.json` file from your Terminal. Name the database `uk_food` and the collection `establishments`.

Within this markdown cell, copy the line of text you used to import the data from your Terminal. This way, future analysts will be able to repeat your process.

e.g.: Import the dataset with mongoimport --type json -d uk_food -c establishments --drop --jsonArray establishments.json

In [21]:
# Import dependencies
from pymongo import MongoClient
from pprint import pprint
import pandas as pd

In [22]:
# Create an instance of MongoClient
mongo = MongoClient(port=27017)

In [23]:
# confirm that our new database was created
print(mongo.list_database_names())

['admin', 'config', 'local', 'uk_food']


In [24]:
# assign the uk_food database to a variable name
uk_food_db = mongo['uk_food']

In [25]:
# review the collections in our new database
print(uk_food_db.list_collection_names())

['establishments']


In [26]:
# review a document in the establishments collection
pprint(uk_food_db.establishments.find_one())


{'AddressLine1': 'Penang Flavours',
 'AddressLine2': '146A Plumstead Rd',
 'AddressLine3': 'London',
 'AddressLine4': '',
 'BusinessName': 'Penang Flavours',
 'BusinessType': 'Restaurant/Cafe/Canteen',
 'BusinessTypeID': 1,
 'Distance': 4623.972328074718,
 'LocalAuthorityCode': '511',
 'LocalAuthorityEmailAddress': 'health@royalgreenwich.gov.uk',
 'LocalAuthorityName': 'Greenwich',
 'LocalAuthorityWebSite': 'http://www.royalgreenwich.gov.uk',
 'NewRatingPending': True,
 'Phone': '',
 'PostCode': 'SE18 7DY',
 'RightToReply': '',
 'SchemeType': 'FHRS',
 '_id': ObjectId('641cac6d5c9488d6e6f7743d'),
 'geocode': {'latitude': Decimal128('51.49014200'),
             'longitude': Decimal128('0.08384000')},
 'scores': {'ConfidenceInManagement': '', 'Hygiene': '', 'Structural': ''}}


In [27]:
# assign the collection to a variable
establishments = uk_food_db['establishments']

## Part 2: Update the Database

1. An exciting new halal restaurant just opened in Greenwich, but hasn't been rated yet. The magazine has asked you to include it in your analysis. Add the following restaurant "Penang Flavours" to the database.

In [28]:
# Create a dictionary for the new restaurant data
new_restaurant ={
    "BusinessName":"Penang Flavours",
    "BusinessType":"Restaurant/Cafe/Canteen",
    "BusinessTypeID":"",
    "AddressLine1":"Penang Flavours",
    "AddressLine2":"146A Plumstead Rd",
    "AddressLine3":"London",
    "AddressLine4":"",
    "PostCode":"SE18 7DY",
    "Phone":"",
    "LocalAuthorityCode":"511",
    "LocalAuthorityName":"Greenwich",
    "LocalAuthorityWebSite":"http://www.royalgreenwich.gov.uk",
    "LocalAuthorityEmailAddress":"health@royalgreenwich.gov.uk",
    "scores":{
        "Hygiene":"",
        "Structural":"",
        "ConfidenceInManagement":""
    },
    "SchemeType":"FHRS",
    "geocode":{
        "longitude":"0.08384000",
        "latitude":"51.49014200"
    },
    "RightToReply":"",
    "Distance":4623.9723280747176,
    "NewRatingPending":True
}

In [29]:
# Insert the new restaurant into the collection
establishments.insert_one(new_restaurant)

In [30]:
# Check that the new restaurant was inserted
query = {'BusinessName': 'Penang Flavours'}
results = establishments.find(query)
for result in results:
    pprint(result)

{'AddressLine1': 'Penang Flavours',
 'AddressLine2': '146A Plumstead Rd',
 'AddressLine3': 'London',
 'AddressLine4': '',
 'BusinessName': 'Penang Flavours',
 'BusinessType': 'Restaurant/Cafe/Canteen',
 'BusinessTypeID': 1,
 'Distance': 4623.972328074718,
 'LocalAuthorityCode': '511',
 'LocalAuthorityEmailAddress': 'health@royalgreenwich.gov.uk',
 'LocalAuthorityName': 'Greenwich',
 'LocalAuthorityWebSite': 'http://www.royalgreenwich.gov.uk',
 'NewRatingPending': True,
 'Phone': '',
 'PostCode': 'SE18 7DY',
 'RightToReply': '',
 'SchemeType': 'FHRS',
 '_id': ObjectId('641cac6d5c9488d6e6f7743d'),
 'geocode': {'latitude': Decimal128('51.49014200'),
             'longitude': Decimal128('0.08384000')},
 'scores': {'ConfidenceInManagement': '', 'Hygiene': '', 'Structural': ''}}
{'AddressLine1': 'Penang Flavours',
 'AddressLine2': '146A Plumstead Rd',
 'AddressLine3': 'London',
 'AddressLine4': '',
 'BusinessName': 'Penang Flavours',
 'BusinessType': 'Restaurant/Cafe/Canteen',
 'BusinessType

2. Find the BusinessTypeID for "Restaurant/Cafe/Canteen" and return only the `BusinessTypeID` and `BusinessType` fields.

In [31]:
# Find the BusinessTypeID for "Restaurant/Cafe/Canteen" and return only the BusinessTypeID and BusinessType fields
query = {'BusinessType': 'Restaurant/Cafe/Canteen'}
fields = {'BusinessTypeID': 1, 'BusinessType': 1}

results = establishments.find(query, fields)

pprint(results[0])

{'BusinessType': 'Restaurant/Cafe/Canteen',
 'BusinessTypeID': 1,
 '_id': ObjectId('641cac6d5c9488d6e6f7743d')}


3. Update the new restaurant with the `BusinessTypeID` you found.

In [32]:
# Update the new restaurant with the correct BusinessTypeID
uk_food_db.establishments.update_one(
    {'BusinessName': 'Penang Flavours'},
    {'$set':
        {'BusinessTypeID': 1}
    }
)

In [33]:
# Confirm that the new restaurant was updated
query = {'BusinessName': 'Penang Flavours'}
fields = {'BusinessName': 1, 'BusinessTypeID': 1}

results = establishments.find(query, fields)
pprint(results[0])


{'BusinessName': 'Penang Flavours',
 'BusinessTypeID': 1,
 '_id': ObjectId('641cac6d5c9488d6e6f7743d')}


4. The magazine is not interested in any establishments in Dover, so check how many documents contain the Dover Local Authority. Then, remove any establishments within the Dover Local Authority from the database, and check the number of documents to ensure they were deleted.

In [34]:
# Find how many documents have LocalAuthorityName as "Dover"
query = {'LocalAuthorityName': 'Dover'}
print('Number of documents with Dover: ', establishments.count_documents(query))

Number of documents with Dover:  0


In [35]:
# Delete all documents where LocalAuthorityName is "Dover"
query = {'LocalAuthorityName': 'Dover'}
establishments.delete_many(query)

In [36]:
# Check if any remaining documents include Dover
query = {'LocalAuthorityName': 'Dover'}
print('Number of remaining documents: ', establishments.count_documents(query))

Number of remaining documents:  0


In [37]:
# Check that other documents remain with 'find_one'
pprint(uk_food_db.establishments.find_one())

{'AddressLine1': 'Penang Flavours',
 'AddressLine2': '146A Plumstead Rd',
 'AddressLine3': 'London',
 'AddressLine4': '',
 'BusinessName': 'Penang Flavours',
 'BusinessType': 'Restaurant/Cafe/Canteen',
 'BusinessTypeID': 1,
 'Distance': 4623.972328074718,
 'LocalAuthorityCode': '511',
 'LocalAuthorityEmailAddress': 'health@royalgreenwich.gov.uk',
 'LocalAuthorityName': 'Greenwich',
 'LocalAuthorityWebSite': 'http://www.royalgreenwich.gov.uk',
 'NewRatingPending': True,
 'Phone': '',
 'PostCode': 'SE18 7DY',
 'RightToReply': '',
 'SchemeType': 'FHRS',
 '_id': ObjectId('641cac6d5c9488d6e6f7743d'),
 'geocode': {'latitude': Decimal128('51.49014200'),
             'longitude': Decimal128('0.08384000')},
 'scores': {'ConfidenceInManagement': '', 'Hygiene': '', 'Structural': ''}}


5. Some of the number values are stored as strings, when they should be stored as numbers. Use `update_many` to convert `latitude` and `longitude` to decimal numbers.

In [38]:
# Change the data type from String to Decimal for longitude
establishments.update_many({}, [{'$set': {'geocode.longitude':\
                                        {'$toDecimal': '$geocode.longitude'}}}])

In [39]:
# Change the data type from String to Decimal for latitude
establishments.update_many({}, [{'$set': {'geocode.latitude':\
                                        {'$toDecimal': '$geocode.latitude'}}}])

In [40]:
# Check that the coordinates are now numbers
document = establishments.find_one()
geocode_dict = document['geocode']

longitude_type = type(geocode_dict['longitude'])
print("The data type of longitude is: ", longitude_type)

latitude_type = type(geocode_dict['latitude'])
print("The data type of latitude is: ", latitude_type)

The data type of longitude is:  <class 'bson.decimal128.Decimal128'>
The data type of latitude is:  <class 'bson.decimal128.Decimal128'>
